In [34]:
import os 
import music21
import music21.instrument
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
# read from melodyData.txt
with open('generated/melodyDataShort.txt', 'r') as f:
    parts = eval(f.read())

# one-hot encoding
encodings = {}
encodingIndex = 0
for part in parts:
    for note in part:
        if note not in encodings:
            encodings[note] = encodingIndex
            encodingIndex += 1
print(len(encodings), "encodings")

decodings = {}
for k, v in encodings.items():
    decodings[v] = k        

# encode everything in indidces first
data_encoded = []
for part in parts:
    data_encoded.append([encodings[note] for note in part])


In [ ]:
print(encodings)

In [37]:
# number of notes per sequence
sequence_length = 10

X = []
Y = []

# given data_encoded, generate training data by looping
for i in range(len(data_encoded)):
    for j in range(len(data_encoded[i]) - sequence_length):
        X.append(data_encoded[i][j:j + sequence_length])
        Y.append(data_encoded[i][j + sequence_length])

# one-hot encode cache
encodings_onehot = {}
# key is note tuple, v is encoding index
for k, v in encodings.items():
    onehot = np.zeros(len(encodings))
    onehot[v] = 1
    encodings_onehot[v] = onehot

# one-hot encode X
X_onehot = []
for seq in X:
    X_onehot.append(np.array([ encodings_onehot[note] for note in seq]))
X = X_onehot

# split this into training and testing sets 
# we will intentionally overfit, GET RID OF THIS LATER
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
# TODO: make the model itself
# benchmark: dense is 80% accuracy
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(sequence_length, len(encodings))),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    # tf.keras.layers.Dropout(.4),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(encodings), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(np.array(X_train), np.array(Y_train), epochs=8)


In [ ]:
loss, accuracy = model.evaluate(np.array(X_test), np.array(Y_test), verbose=1)
print(f'Loss: {loss}, Accuracy: {accuracy}')

In [ ]:
import copy
# given a sequence, generate the next note (up to ten times)
seq = copy.deepcopy(X_train[1])
total_seq = seq[:]
for i in range(10):
    pred = model.predict(np.array([seq]))
    pred = np.argmax(pred)
    # append to sequence the one-hot encoding of pred
    one_hot = np.zeros((len(encodings)))
    one_hot[pred] = 1
    total_seq = np.append(total_seq, [one_hot], axis=0)
    # set sequence to be the last ten values of total_seq
    seq = total_seq[-10:]
    #print (seq)

    #total_seq.append(pred)
    # total seq is a numpy array, can't just append things to it

mapping = {0: "C", 1: "C#", 2: "D", 3: "D#", 4: "E", 5: "F", 6: "F#", 7: "G", 8: "G#", 9: "A", 10: "A#", 11: "B"}
    
for note in total_seq:
    pitch, dur = decodings[np.argmax(note)]
    print (mapping[(pitch - 1) % 12] if pitch != 0 else "Rest", dur)

# convert numbers to musical note letters
# 60 = C